In [2]:
import numpy as np

from sklearn.datasets import make_regression

# Base Regression Model

In [3]:
class RegressionModel:
    def __init__(self):
        self.coef = None
        self.bias = None
        self.w = None

    def fit(self, X: np.ndarray, y: np.ndarray):
        raise NotImplementedError('Subclasses should implement this class')
    
    def predict(self, X: np.ndarray):
        pass

    def add_bias(self, X: np.ndarray):
        n_samples = X.shape[0]
        bias_term = np.ones((n_samples, 1))
        X_bias = np.hstack((bias_term, X))

        return X_bias
    
    def score(self, X: np.ndarray, y: np.ndarray):
        predictions = self.predict(X)
        ss_total = np.sum((y - np.mean(y)) ** 2)
        ss_residual = np.sum((y - predictions) ** 2)
        return 1 - (ss_residual / ss_total)

### A Regression model trained by normal Equation

In [4]:
class NormalRegression(RegressionModel):
    def __init__(self):
        super().__init__()

    def fit(self, X: np.ndarray, y: np.ndarray):
        X = self.add_bias(X)

        self.w = np.linalg.pinv(X.T @ X) @ X.T @ y
    
    def predict(self, X):
        X = self.add_bias(X)
        return X @ self.w

In [13]:
class LinearRegression(RegressionModel):
    def __init__(self, epoches=1000, learning_rate=0.1, lr_type='batch', batch_size=None):
        super().__init__()
        self.epoches = epoches
        self.learning_rate = learning_rate
        self.n_samples = 0
        self.type = lr_type
        self.batch_size = batch_size

    def fit(self, X: np.ndarray, y: np.ndarray):
        self.n_samples, n_features = X.shape[0], X.shape[1]
        self.w = np.random.randn(n_features + 1) * 0.01
        X_bias = self.add_bias(X)

        if self.type == 'batch':
            self.fit_batch(X_bias, y)
        elif self.type == 'stochastic':
            self.fit_stochastic(X_bias, y)
        elif self.type == 'mini-batch':
            self.fit_mini_batch(X_bias, y)

    def fit_batch(self, X: np.ndarray, y: np.ndarray):
        for _ in range(self.epoches):
            predictions = X @ self.w
            errors = predictions - y
            gradient = (X.T @ errors) / self.n_samples

            self.w = self.w - self.learning_rate * gradient

    def fit_stochastic(self, X: np.ndarray, y: np.ndarray):
        for _ in range(self.epoches):
            random_idx = np.random.randint(self.n_samples)
            random_sample = X[random_idx]
            predict = self.w.T.dot(random_sample)
            error = predict - y[random_idx]

            stoch = error * random_sample
            self.w = self.w - self.learning_rate * stoch

    def fit_mini_batch(self, X: np.ndarray, y: np.ndarray):
        for _ in range(self.epoches):
            size = self.batch_size if self.batch_size else max(1, int(self.n_samples * 0.1))
            random_inds = np.random.randint(0, self.n_samples, size=size)

            batch_samples, batch_labels = X[random_inds], y[random_inds]
            predictions = batch_samples @ self.w
            errors = predictions - batch_labels

            gradient = ( batch_samples.T @ errors ) / size

            self.w = self.w - self.learning_rate * gradient

    def predict(self, X:np.ndarray):
        X = self.add_bias(X)
        return X @ self.w

In [14]:
X, y = make_regression(1000, 2)

lr = LinearRegression(lr_type='mini-batch')
lr.fit(X, y)

In [15]:
lr.score(X, y)

1.0